In [ ]:
import sys
from pathlib import Path

# The correct way to get the project root.
project_root = Path().resolve().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"Project root added to path: {project_root}")

Project root added to path: C:\Users\Willard\python-projects\emoji_sentiment_analysis


In [2]:
# block 1

import pandas as pd
import joblib
import ipywidgets as widgets
from IPython.display import display, clear_output
import re
from emoji_sentiment_analysis.config import MODELS_DIR

# Regex to find emojis
EMOJI_PATTERN = re.compile(
    "["
    "\U0001F600-\U0001F64F"  # Emoticons
    "\U0001F300-\U0001F5FF"  # Symbols & Pictographs
    "\U0001F680-\U0001F6FF"  # Transport & Map Symbols
    "\U0001F1E0-\U0001F1FF"  # Flags (iOS)
    "\U00002702-\U000027B0"  # Dingbats
    "]+",
    flags=re.UNICODE
)

def extract_emojis(text):
    return " ".join(re.findall(EMOJI_PATTERN, text))

# Load the trained model and vectorizer
model = joblib.load(MODELS_DIR / "sentiment_model.pkl")
vectorizer = joblib.load(MODELS_DIR / "tfidf_vectorizer.pkl")

print("Sentiment analyzer loaded successfully! 🚀")

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Willard\\python-projects\\emoji_sentiment_analysis\\models\\sentiment_model.pkl'

In [ ]:
# block 2

# Create the text input and button widgets
text_input = widgets.Textarea(
    description='Enter a sentence:',
    placeholder='I love sentiment analysis but it is too hard to study 😔'
)
sentiment_button = widgets.Button(
    description='Say your Sentiments!',
    button_style='success'
)
output_area = widgets.Output()

# Display the widgets
display(text_input, sentiment_button, 
output_area)

Textarea(value='', description='Enter a sentence:', placeholder='I love sentiment analysis but it is too hard …

Button(button_style='success', description='Say your Sentiments!', style=ButtonStyle())

Output()

In [ ]:
# block 3

def on_button_click(b):
    with output_area:
        clear_output()  # Clear previous output
        user_input = text_input.value
        
        if not user_input:
            print("Please enter a sentence to analyze.")
            return

        print("==========================================================")
        print(f'Your input is: "{user_input}"')

        # Preprocess the user input, including amplifying emojis
        processed_input = user_input + ' ' + extract_emojis(user_input) * 5
        
        # Transform the text using the trained vectorizer
        transformed_input = vectorizer.transform([processed_input])
        
        # Predict the sentiment
        prediction = model.predict(transformed_input)[0]
        
        # Display the result
        print("==========================================================")
        print(f'YOUR INPUT IS OF "{prediction.upper()}" SENTIMENT')
        print("==========================================================")

# Link the button's on_click event to the analysis function
sentiment_button.on_click(on_button_click)